Construindo um grafo automaticamente
====================================



## Introdução



Neste notebook nós vamos dar o primeiro passo para construir nossa rede neural artificial. Neste primeiro passo, nós vamos criar uma classe que gera automaticamente o nosso `grafo computacional`. O grafo computacional é o grafo que representa todas as operações matemáticas que ocorreram ao se computar um certo valor $y$. O grafo computacional é um passo necessário pois será baseado nele que iremos computar os gradientes locais necessários para realizar o `backpropagation`.



## Importações



In [14]:
from graphviz import Digraph
from funcoes import plota_grafo

## Código e discussão



### Primeiros passos



A base de tudo será uma classe chamada `Valor`. Vamos começar pelo básico!



In [15]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

Vamos testar nossa classe!



In [16]:
x1 = Valor(60)
print(x1)
print(x1.data)

Valor(data=60)
60


### Os dunders `__add__` e `__mul__`



Observe que não conseguimos adicionar nem multiplicar objetos criados com a classe `Valor`.



In [17]:
a = Valor(10)
b = Valor(5)

In [18]:
print(a + b)

TypeError: unsupported operand type(s) for +: 'Valor' and 'Valor'

In [19]:
print(a * b)

TypeError: unsupported operand type(s) for *: 'Valor' and 'Valor'

Ué&#x2026; porque não conseguimos? Não conseguimos pois o Python (ainda) não é vidente. Ele lá vai saber como adicionar ou multiplicar algo que você criou? Pra você parece óbvio que valores podem ser adicionados ou multiplicados, mas para o Python ele nem sabe o que significa a palavra `Valor`&#x2026;

Como sempre, temos que contar para o programa o que queremos que aconteça quando usarmos os operadores `+` e `*`. Quem faz isso são os dunders `__add__` e `__mul__`.



In [20]:
class Valor:
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        saida = Valor(self.data + outro_valor.data)
        return saida

    def __mul__(self, outro_valor):
        saida = Valor(self.data * outro_valor.data)
        return saida

Vamos testar!



In [21]:
c = Valor(15)
d = Valor(10)
e = c + d
f = c * d
print(e, f)

Valor(data=25) Valor(data=150)


### Registrando os progenitores



Nosso objetivo é construir um grafo computacional. Em um grafo computacional, um certo vértice pode ter um ou mais vértices progenitores (são seus pais/mães). Nós não podemos perder essa informação quando formos construir um grafo, então precisamos incluir essa informação na nossa classe.



In [22]:
class Valor:
    def __init__(self, data, progenitor=()):
        self.data = data
        self.progenitor = progenitor

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        saida = Valor(data, progenitor)
        return saida

Vamos testar!



In [23]:
a = Valor(10)
c = a + a

print(a.data, a.progenitor)
print(c.data, c.progenitor)

10 ()
20 (Valor(data=10), Valor(data=10))


### Registrando o operador mãe



Em um grafo computacional, um vértice pode ter um operador mãe. O operador mãe é o operador que foi usado para gerar o vértice.



In [24]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [25]:
a = Valor(10)
c = a + a
d = a*a

print(a.data, a.progenitor, a.operador_mae)
print(c.data, c.progenitor, c.operador_mae)
print(d.data, d.progenitor, d.operador_mae)

10 () 
20 (Valor(data=10), Valor(data=10)) +
100 (Valor(data=10), Valor(data=10)) *


### Plotando o primeiro grafo



Vamos plotar nosso primeiro grafo!



In [29]:
a = Valor(2)
b = Valor(-3)
c = Valor(10)

d = a * b
e = d + c

g = plota_grafo(e)
print(g)

digraph {
	graph [rankdir=LR]
	2502079269472 [label="{  | data 10.0000 }" shape=record]
	2502079237232 [label="{  | data -3.0000 }" shape=record]
	2502079270096 [label="{  | data 4.0000 }" shape=record]
	"2502079270096+" [label="+"]
	"2502079270096+" -> 2502079270096
	2502079270240 [label="{  | data 2.0000 }" shape=record]
	2502079267696 [label="{  | data -6.0000 }" shape=record]
	"2502079267696*" [label="*"]
	"2502079267696*" -> 2502079267696
	2502079269472 -> "2502079270096+"
	2502079270240 -> "2502079267696*"
	2502079267696 -> "2502079270096+"
	2502079237232 -> "2502079267696*"
}



### Registrando o rótulo



Nosso grafo seria mais legível se tivéssemos rótulos indicando o que é cada vértice. Vamos incluir essa informação na nossa classe.



In [27]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo

    def __repr__(self):
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        data = self.data + outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "+"
        saida = Valor(data, progenitor, operador_mae)
        return saida

    def __mul__(self, outro_valor):
        data = self.data * outro_valor.data
        progenitor = (self, outro_valor)
        operador_mae = "*"
        saida = Valor(data, progenitor, operador_mae)
        return saida

Vamos testar!



In [30]:
a = Valor(2, rotulo="a")
b = Valor(-3, rotulo="b")
c = Valor(10, rotulo="c")

d = a * b
e = d + c

d.rotulo = "d"
e.rotulo = "e"

g = plota_grafo(e)
print(g)

digraph {
	graph [rankdir=LR]
	2502079267888 [label="{ c | data 10.0000 }" shape=record]
	2502079269616 [label="{ e | data 4.0000 }" shape=record]
	"2502079269616+" [label="+"]
	"2502079269616+" -> 2502079269616
	2502078557968 [label="{ a | data 2.0000 }" shape=record]
	2502079267744 [label="{ d | data -6.0000 }" shape=record]
	"2502079267744*" [label="*"]
	"2502079267744*" -> 2502079267744
	2502078555616 [label="{ b | data -3.0000 }" shape=record]
	2502078557968 -> "2502079267744*"
	2502079267888 -> "2502079269616+"
	2502078555616 -> "2502079267744*"
	2502079267744 -> "2502079269616+"
}



### Refazendo o grafo que fizemos na aula anterior



Na aula anterior nós fizemos um grafo computacional para aprender como funciona o backpropagation. Vamos refazer ele aqui!



In [33]:
x1 = Valor(60, rotulo='x1')
x2 = Valor(24, rotulo='x2')
w1 = Valor(10, rotulo='w1')
w2 = Valor(5, rotulo='w2')
w3 = Valor(2, rotulo='w3')
b = Valor(7, rotulo='b')

s1 = x1*w1
s1.rotulo = 's1'
s2 = x2*w2
s2.rotulo = 's2'

n = s1+s2
n.rotulo = 'n'

k = n+b
k.rotulo = 'k'

y = k*w3
y.rotulo = 'y'

g = plota_grafo(y)
print(g)

digraph {
	graph [rankdir=LR]
	2502079236128 [label="{ b | data 7.0000 }" shape=record]
	2502079234208 [label="{ w2 | data 5.0000 }" shape=record]
	2502079234256 [label="{ s2 | data 120.0000 }" shape=record]
	"2502079234256*" [label="*"]
	"2502079234256*" -> 2502079234256
	2502079236320 [label="{ x2 | data 24.0000 }" shape=record]
	2502079757568 [label="{ y | data 1454.0000 }" shape=record]
	"2502079757568*" [label="*"]
	"2502079757568*" -> 2502079757568
	2502079234352 [label="{ x1 | data 60.0000 }" shape=record]
	2502079237952 [label="{ s1 | data 600.0000 }" shape=record]
	"2502079237952*" [label="*"]
	"2502079237952*" -> 2502079237952
	2502079234880 [label="{ w3 | data 2.0000 }" shape=record]
	2502079757664 [label="{ k | data 727.0000 }" shape=record]
	"2502079757664+" [label="+"]
	"2502079757664+" -> 2502079757664
	2502079234400 [label="{ w1 | data 10.0000 }" shape=record]
	2502079235024 [label="{ n | data 720.0000 }" shape=record]
	"2502079235024+" [label="+"]
	"2502079235024+" -> 

## Conclusão



## Playground

